# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
import torch
from torchvision import datasets, transforms, models
from torch import nn, optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
training_data_transforms = transforms.Compose([transforms.RandomRotation(30),
                                               transforms.RandomResizedCrop(224), 
                                               transforms.RandomHorizontalFlip(), 
                                               transforms.ToTensor(), 
                                               transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

validaiton_data_transforms = transforms.Compose([transforms.Resize(256),
                                                 transforms.CenterCrop(224),
                                                 transforms.ToTensor(), 
                                                 transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

test_data_transforms = transforms.Compose([transforms.Resize(256),
                                           transforms.CenterCrop(224),
                                           transforms.ToTensor(), 
                                           transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
# TODO: Load the datasets with ImageFolder
train_image_dataset = datasets.ImageFolder(train_dir, transform = training_data_transforms)
valid_image_dataset = datasets.ImageFolder(valid_dir, transform = validaiton_data_transforms)
test_image_dataset = datasets.ImageFolder(test_dir, transform = test_data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_dataloader = torch.utils.data.DataLoader(train_image_dataset, batch_size = 32, shuffle = True)
valid_dataloader = torch.utils.data.DataLoader(valid_image_dataset, batch_size = 32, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_image_dataset, batch_size = 32, shuffle = True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [5]:
model = models.vgg16(pretrained=True)
model

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:26<00:00, 20876961.57it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [6]:
# TODO: Build and train your network
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# TODO: Build and train your network
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

In [8]:
# TODO: Build and train your network
#Define our new classifier which we will be training
from collections import OrderedDict
classifier = nn.Sequential(nn.Linear(25088, 1500),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(1500,102),
                      nn.LogSoftmax(dim=1))

In [9]:
# TODO: Build and train your network
#We have created our classifier, now let's attach this to our model
model.classifier = classifier

In [10]:
# TODO: Build and train your network
#let's check our model now is the way we want it, thus the last classifier for the CNN VGG network is the way we defined it
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [11]:
# TODO: Build and train your network
#Here I define my loss
#Since our last laye is LogSoftmax we use Negative Log Likelihood loss function NLLLoss()
criterion = nn.NLLLoss()

In [17]:
# TODO: Build and train your network
#Here I defime my optimizer
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.SGD(model.classifier.parameters(), lr=0.01)

In [13]:
# TODO: Build and train your network
#move the model to whichever device we have available
model.to(device);

In [18]:
# TODO: Build and train your network
#Now we have the model setup so we are ready to train it

#Define variables we will be using during the training
epochs = 5 
steps = 0 #we will track the number of trainsteps we do so we set this to zero
training_loss = 0 #we wil track the training loss we do so we set this to zero
print_every = 5
for epoch in range(epochs):
    for images, labels in train_dataloader: #loop into our data (images)
        steps += 1 #cumulate steps. Every time we go through one of these batches, we will increment steps here
        images, labels = images.to(device), labels.to(device)# Move input and label tensors to the default device
        optimizer.zero_grad() #Here we zero our gradients in order not to have leftover values from previous loops
        
        log_ps = model.forward(images) #get our log probabilities from our model
        
        loss = criterion(log_ps, labels) #We get the loss from the criterion
        loss.backward() #We do a backwards pass
        optimizer.step() #With the optimizer we take a step
        training_loss += loss.item() #Here we increment our training loss - This is where we keep track of our training loss
               
        ### This is where our training loop ends ###
        ### Now we drop out of our training loop to test our network's accuracy and loss on our validation dataset ###
        
        
        
        if steps % print_every == 0:
            valid_loss = 0
            accuracy = 0
            model.eval() # this turns our model to the evaluation inference mode which turns off Dropout
            # In this way we can accurately use our network for making predictions

            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                for images, labels in valid_dataloader:
                    images, labels = images.to(device), labels.to(device) #transfer my tensors to the GPU
                    log_ps = model(images) #with the model we already have we will pass images from our validation dataset
                    valid_loss += criterion(log_ps, labels) #get loss from validation dataset
                
                    #calculate accuracy
                    ps = torch.exp(log_ps) #our model returns log probabilities, so to get crude probabilities we do this
                    top_p, top_class = ps.topk(1, dim=1) #here we get top probabilities and top classes using ps.topk()
                    equality = top_class == labels.view(*top_class.shape) #check for equality with our labels
                    accuracy += torch.mean(equality.type(torch.FloatTensor)) #with equality tensor we update our accuracy

            print(f"Epoch {epoch+1}/{epochs}.. "
                f"Train loss: {training_loss/print_every:.3f}.. "
                f"Valid loss: {valid_loss/len(valid_dataloader):.3f}.. "
                f"Valid accuracy: {accuracy/len(valid_dataloader):.3f}")
                
            training_loss = 0 #we set our training loss back to zero
            model.train() #we set our model in training mode - we now use the dropout

Epoch 1/5.. Train loss: 3.784.. Test loss: 3.481.. Test accuracy: 0.230
Epoch 1/5.. Train loss: 3.741.. Test loss: 3.408.. Test accuracy: 0.268
Epoch 1/5.. Train loss: 3.620.. Test loss: 3.272.. Test accuracy: 0.285
Epoch 1/5.. Train loss: 3.500.. Test loss: 3.109.. Test accuracy: 0.403
Epoch 1/5.. Train loss: 3.599.. Test loss: 2.963.. Test accuracy: 0.379
Epoch 1/5.. Train loss: 3.392.. Test loss: 2.963.. Test accuracy: 0.349
Epoch 1/5.. Train loss: 3.245.. Test loss: 2.901.. Test accuracy: 0.399
Epoch 1/5.. Train loss: 3.005.. Test loss: 2.773.. Test accuracy: 0.385
Epoch 1/5.. Train loss: 3.275.. Test loss: 2.736.. Test accuracy: 0.395
Epoch 1/5.. Train loss: 2.999.. Test loss: 2.580.. Test accuracy: 0.497
Epoch 1/5.. Train loss: 3.066.. Test loss: 2.503.. Test accuracy: 0.489
Epoch 1/5.. Train loss: 2.891.. Test loss: 2.355.. Test accuracy: 0.532
Epoch 1/5.. Train loss: 2.851.. Test loss: 2.294.. Test accuracy: 0.517
Epoch 1/5.. Train loss: 2.602.. Test loss: 2.262.. Test accuracy

In [ ]:
images.shape

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [19]:
# TODO: Do validation on the test set
test_loss = 0
accuracy = 0
print_every = 5
model.eval() # this turns our model to the evaluation inference mode which turns off Dropout
            # In this way we can accurately use our network for making predictions
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device) #transfer my tensors to the GPU
        log_ps = model(images) #with the model we already have we will pass images from our validation dataset
        test_loss += criterion(log_ps, labels) #get loss from validation dataset
                
        #calculate accuracy
        ps = torch.exp(log_ps) #our model returns log probabilities, so to get crude probabilities we do this
        top_p, top_class = ps.topk(1, dim=1) #here we get top probabilities and top classes using ps.topk()
        equality = top_class == labels.view(*top_class.shape) #check for equality with our labels
        accuracy += torch.mean(equality.type(torch.FloatTensor)) #with equality tensor we update our accuracy
                

        print(f"Train loss: {test_loss/print_every:.3f}.. "
                f"Test loss: {test_loss/len(test_dataloader):.3f}.. "
                f"Test accuracy: {accuracy/len(test_dataloader):.3f}")

Train loss: 0.099.. Test loss: 0.019.. Test accuracy: 0.032
Train loss: 0.185.. Test loss: 0.036.. Test accuracy: 0.067
Train loss: 0.278.. Test loss: 0.054.. Test accuracy: 0.100
Train loss: 0.342.. Test loss: 0.066.. Test accuracy: 0.135
Train loss: 0.448.. Test loss: 0.086.. Test accuracy: 0.168
Train loss: 0.498.. Test loss: 0.096.. Test accuracy: 0.204
Train loss: 0.571.. Test loss: 0.110.. Test accuracy: 0.238
Train loss: 0.683.. Test loss: 0.131.. Test accuracy: 0.270
Train loss: 0.762.. Test loss: 0.147.. Test accuracy: 0.306
Train loss: 0.806.. Test loss: 0.155.. Test accuracy: 0.343
Train loss: 0.907.. Test loss: 0.174.. Test accuracy: 0.377
Train loss: 1.016.. Test loss: 0.195.. Test accuracy: 0.411
Train loss: 1.159.. Test loss: 0.223.. Test accuracy: 0.441
Train loss: 1.338.. Test loss: 0.257.. Test accuracy: 0.471
Train loss: 1.392.. Test loss: 0.268.. Test accuracy: 0.508
Train loss: 1.508.. Test loss: 0.290.. Test accuracy: 0.541
Train loss: 1.648.. Test loss: 0.317.. T

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
#Now that we have our network trained, we can save it to a file, what's called the state_dict
#state_dict is a dictionary that contains all the parameters for our model - all weights and bias tensors.

#print out our model
print("Our model: \n\n", model, "\n")
print("The state dict keys: \n\n", model.state_dict().keys(), "\n")

In [ ]:
#save the state_dict() with torch.save to a checkpoint.pth
model.class_to_idx = image_datasets['train'].class_to_idx
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'mapping_of_classes_to_indices': model.class_to_idx,
            'loss': loss,
            }, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def 

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes